Author: Lai Khee Jiunn
  
  

Source: 

    [1] http://www.bursamalaysia.com/market/listed-companies/list-of-companies/main-market/
    [2] https://www.morningstar.com/
    [3] https://www.klsescreener.com/
    [4] https://klse.i3investor.com/
  

Reference:

    [1] K. M. Ho, "How to make money from your stock investment even in a falling market", 3rd ed., Malaysia, Kanyin Publications Sdn. Bhd., 2017.

Note: The entire process in this jupyter notebook may take up to 2.5 hours depending on your internet speed

In [1]:
import numpy as np
import pandas as pd
import re, time, os
from datetime import datetime, date
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
# from time import time
from tqdm import tqdm
from glob import glob
chromedriver = 'D://Workspace/selenium_driver/bin/chromedriver'
out_dir = 'result_2020-05-17/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [2]:
stock_code = pd.read_csv(out_dir+'klse_00_stock_code.csv')
data1_1 = pd.read_csv(out_dir+'klse_01a_latest_data.csv')
data1_2 = pd.read_csv(out_dir+'klse_01b_latest_data.csv')
data1 = pd.read_csv(out_dir+'klse_01_latest_data.csv')
data2 = pd.read_csv(out_dir+'klse_02_10yr_data.csv')
data3 = pd.read_csv(out_dir+'klse_03_stage_3.csv')
# data4 = pd.read_csv(out_dir+'klse_04_intrinsic.csv')
# data5 = pd.read_csv(out_dir+'klse_05_3G2H_score_with_valuation.csv')
# final = pd.read_csv(out_dir+'klse_06_final_table.csv')

data1_1['code'] = data1_1['code'].astype(str).apply(lambda x: x.zfill(4))
data1_2['code'] = data1_2['code'].astype(str).apply(lambda x: x.zfill(4))
data1['code'] = data1['code'].astype(str).apply(lambda x: x.zfill(4))
data2['code'] = data2['code'].astype(str).apply(lambda x: x.zfill(4))
data3['code'] = data3['code'].astype(str).apply(lambda x: x.zfill(4))
# data4['code'] = data4['code'].astype(str).apply(lambda x: x.zfill(4))
# data5['code'] = data5['code'].astype(str).apply(lambda x: x.zfill(4))
# final['code'] = final['code'].astype(str).apply(lambda x: x.zfill(4))

### Get all listed company names and stock codes from Bursa Malaysia

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
driver = webdriver.Chrome(chromedriver, options=options)
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
driver.get('http://www.bursamalaysia.com/market/listed-companies/list-of-companies/main-market/')
driver.find_element_by_class_name('cc-btn').click()
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight + 200);")
time.sleep(2)
driver.find_element_by_class_name('custom-select').click()
driver.find_element_by_class_name('custom-select').find_elements_by_tag_name('option')[-1].click()
time.sleep(2)
x = driver.find_elements_by_class_name('listed-company-table')[-1]\
          .find_element_by_tag_name('tbody')\
          .find_elements_by_class_name('company-announcement-link')

code, name = [],[]
for i in tqdm(x):
    code.append(i.get_attribute('href').split('=')[-1])
    name.append(i.get_attribute('innerText').strip())
driver.quit()

100%|████████████████████████████████████████████████████████████████████████████| 788/788 [00:06<00:00, 112.91it/s]


In [4]:
stock_code = pd.DataFrame({'code':code,'name':name})
idx1 = stock_code[stock_code.name == 'KLCC REAL ESTATE INVESTMENT TRUST'].index
idx2 = stock_code[stock_code.name == 'KLCC PROPERTY HOLDINGS BERHAD'].index
stock_code.loc[idx1, 'code'] = '5235SS'
stock_code.drop(index=idx2,inplace=True)
stock_code.reset_index(drop=True)
print(stock_code.shape)
stock_code.to_csv(out_dir+'klse_00_stock_code.csv', index=False)
stock_code.head(1)

(788, 2)


,code,name
0,5250,7-ELEVEN MALAYSIA HOLDINGS BERHAD


### Stage 1 & 2

In [167]:
def process_stage_1(stock_code):
    # Get stock prices in 52 weeks range from KLSE Screener, which will be used in Stage 4
    start_time = time.time()
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    driver = webdriver.Chrome(chromedriver, options=options)
    driver.switch_to.window(driver.current_window_handle)
    driver.maximize_window()
    error1, result1 = [],[]
    for i in tqdm(stock_code.code):
        try:
            driver.get('https://www.klsescreener.com/v2/stocks/view/'+i)
#             time.sleep(1)
            try:
                cat= driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/span[2]')\
                           .get_attribute('innerText')\
                           .split(' - ')[0]
            except:
                cat= 'UNKNOWN'
            price  = driver.find_element_by_id('price')\
                           .get_attribute('innerText')
            week52 = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[7]/td[2]')\
                           .get_attribute('innerText')\
                           .split(' - ')
            roe    = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[8]/td[2]')\
                           .get_attribute('innerText')\
                           .replace(',','')
            pe     = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[9]/td[2]')\
                           .get_attribute('innerText')
            eps    = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[10]/td[2]')\
                           .get_attribute('innerText')
            ey     = np.around(float(eps)/float(price), 2)
            dps    = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[11]/td[2]')\
                           .get_attribute('innerText')
            dy     = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[12]/td[2]')\
                           .get_attribute('innerText')\
                           .replace('%','')
            bvps   = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[14]/td[2]')\
                           .get_attribute('innerText')
#             cap    = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[18]/td[2]')\
#                            .get_attribute('innerText')

            if len(roe)==0:
                roe = 0.0

            result1.append([str(i).zfill(4), str(cat), float(price), float(week52[0]), float(week52[-1]), float(bvps),
                            float(pe), float(eps), ey, float(dps), float(dy), float(roe)])
        except Exception as e:
            error1.append([str(i).zfill(4), type(e).__name__])
    driver.quit()
    print('Time taken: ',time.time()-start_time)
    data1 = pd.DataFrame(result1, columns=['code','category','Price','52w low','52w high','BVPS',
                                           'P/E','EPS','EY','DPS','DY','ROE'])
    er_df_1 = pd.DataFrame(error1, columns=['code','reason'])
    return data1, er_df_1

In [40]:
def process_stage_2(balance, offset=0):
    start_time = time.time()
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    driver = webdriver.Chrome(chromedriver, options=options)
    driver.switch_to.window(driver.current_window_handle)
    driver.maximize_window()
    symbol = ['—']
    
    driver.get('https://financials.morningstar.com/ratios/r.html?t=5398&culture=en&platform=sal&region=mys')
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="orderType"]').click()
    driver.find_element_by_xpath('//*[@id="Li2"]/ul/li[2]/a').click()
#     driver.execute_script('''window.open("","_blank");''')    
#     driver.switch_to.window(driver.window_handles[0])

    size  = 50
    batch = int(np.ceil(len(balance)/size))
    error1_2, error2 = [], []
    for b in range(batch):
        result1_2, data2 = [], pd.DataFrame()
        for i in tqdm(balance[b*size:(b+1)*size].code):
            try:
                driver.get('https://financials.morningstar.com/ratios/r.html?t='+str(i)+'&culture=en&platform=sal&region=mys')
                time.sleep(2)
                year_end = [str(i), 'Year End'] + driver.find_element_by_id('financials').find_element_by_tag_name('thead')\
                                                        .get_attribute('innerText').split('\n')[0].split('\t')[2:]
                assert len(year_end) == 12, 'Number of year mismatch'

                table_1 = driver.find_element_by_id('financials').find_element_by_tag_name('tbody')\
                                .find_elements_by_tag_name('tr')[1::2]

                revenue    = [str(i), 'Revenue'] + table_1[0].get_attribute('innerText').split('\t')[2:]
                net_profit = [str(i), 'NP'] + table_1[4].get_attribute('innerText').split('\t')[2:]
                eps        = [str(i), 'EPS'] + table_1[5].get_attribute('innerText').split('\t')[2:]
                dps        = [str(i), 'DPS'] + table_1[6].get_attribute('innerText').split('\t')[2:]
                div_payout = [str(i), 'Div Payout'] + table_1[7].get_attribute('innerText').split('\t')[2:]
                shares     = [str(i), 'Shares'] + table_1[8].get_attribute('innerText').split('\t')[2:]
                bvps        = [str(i), 'BVPS'] + table_1[9].get_attribute('innerText').split('\t')[2:]
                ocf        = [str(i), 'OCF'] + table_1[10].get_attribute('innerText').split('\t')[2:]

                tabs = driver.find_element_by_class_name('in_tabs').find_elements_by_tag_name('li')

                tabs[0].click()
                table_2 = driver.find_element_by_id('tab-profitability').find_elements_by_tag_name('table')[-1]\
                                .find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')[1::2]

                np_margin  = [str(i), 'NP Margin'] + table_2[1].get_attribute('innerText').split('\t')[2:]
                temp       = table_2[5].get_attribute('innerText').split('\t')
                roe        = [str(i), 'ROE'] + temp[2:]
    #             roe_last   = temp[1].replace(',','') if temp[1] not in symbol else 0.0
                roe_last2  = temp[2].replace(',','') if temp[2] not in symbol else 0.0

                tabs[1].click()
                table_3 = driver.find_element_by_id('tab-growth').find_element_by_tag_name('table')\
                                .find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')[1::2]

                rev_gr_yoy     = [str(i), 'Revenue GR'] + table_3[1].get_attribute('innerText').split('\t')[2:]
                rev_gr_3y_avg  = [str(i), 'Revenue GR 3Y avg'] + table_3[2].get_attribute('innerText').split('\t')[2:]
                rev_gr_5y_avg  = [str(i), 'Revenue GR 5Y avg'] + table_3[3].get_attribute('innerText').split('\t')[2:]
                rev_gr_10y_avg = [str(i), 'Revenue GR 10Y avg'] + table_3[4].get_attribute('innerText').split('\t')[2:]

                eps_gr_yoy     = [str(i), 'EPS GR'] + table_3[-4].get_attribute('innerText').split('\t')[2:]
                eps_gr_3y_avg  = [str(i), 'EPS GR 3Y avg'] + table_3[-3].get_attribute('innerText').split('\t')[2:]
                eps_gr_5y_avg  = [str(i), 'EPS GR 5Y avg'] + table_3[-2].get_attribute('innerText').split('\t')[2:]
                eps_gr_10y_avg = [str(i), 'EPS GR 10Y avg'] + table_3[-1].get_attribute('innerText').split('\t')[2:]

                tabs[2].click()
                table_4 = driver.find_element_by_id('tab-cashflow').find_element_by_tag_name('table')\
                                .find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')[1::2]

                ocf_gr_yoy = [str(i), 'OCF GR'] + table_4[0].get_attribute('innerText').split('\t')[2:]

                tabs[3].click()
                table_5 = driver.find_element_by_id('tab-financial').find_elements_by_tag_name('table')[-1]\
                                .find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')[1::2]

                temp       = table_5[-1].get_attribute('innerText').split('\t')
                de         = [str(i), 'D/E'] + temp[2:]
                de_last    = temp[1].replace(',','') if temp[1] not in symbol else 0.0
                de_last2   = temp[2].replace(',','') if temp[2] not in symbol else 0.0

                try:
                    driver.get('https://klse.i3investor.com/servlets/stk/fin/'+str(i)+'.jsp?type=last10fy')
                    time.sleep(2)
                    x = driver.find_elements_by_tag_name('table')[13].find_elements_by_tag_name('tr')
                    if len(x) != 119:
                        x = driver.find_elements_by_tag_name('table')[14].find_elements_by_tag_name('tr')
                    price  = [str(i), 'Price'] + [x.get_attribute('innerText') for x in x[84].find_elements_by_tag_name('td')[3:-1]]
                except Exception as e:
                    error1_2.append([str(i), type(e).__name__])

                result1_2.append([str(i).zfill(4), roe_last2, de_last, de_last2])
                temp_data  = [year_end, revenue, net_profit, eps, dps, div_payout, shares, bvps, ocf,
                              np_margin, roe, rev_gr_yoy, rev_gr_3y_avg, rev_gr_5y_avg, rev_gr_10y_avg, 
                              eps_gr_yoy, eps_gr_3y_avg, eps_gr_5y_avg, eps_gr_10y_avg, ocf_gr_yoy, de, price]
                data2      = pd.concat([data2, pd.DataFrame(temp_data)])
            except Exception as e:
                error2.append([i, type(e).__name__])
                
        data1_2 = pd.DataFrame(result1_2, columns=['code','ROE2','D/E','D/E2'])
        data2.columns = ['code','detail','Y1','Y2','Y3','Y4','Y5','Y6','Y7','Y8','Y9','Y10']
        data2.fillna(0.0, inplace=True)
        
        data1_2.to_csv('temp_data/data1_2_part'+str(b+offset).zfill(2)+'.csv', index=False)
        data2.to_csv('temp_data/data2_part'+str(b+offset).zfill(2)+'.csv', index=False)        
#         result1_2, data2 = [], pd.DataFrame()
        
    driver.quit()
    print('Time taken: ',time.time()-start_time)
    
    er_df_1 = pd.DataFrame(error1_2, columns=['code','reason'])
    er_df_2 = pd.DataFrame(error2, columns=['code','reason'])
    
    return er_df_1, er_df_2

In [ ]:
data1_1, er_df_1_1 = process_stage_1(stock_code)
print(data1_1.shape)
data1_1.to_csv(out_dir+'klse_01a_latest_data.csv', index=False)
display(data1_1.head(1))
display(er_df_1_1.reason.value_counts())
assert(len(er_df_1_1) == 0)

In [15]:
er_df_1_1

,code,reason
0,5194,NoSuchElementException


In [41]:
er_df_1_2, er_df_2 = process_stage_2(data1_1)
display(er_df_1_2.reason.value_counts())
display(er_df_2.reason.value_counts())

assert(len(er_df_1_2) == 0) 
assert(len(er_df_2) == 0) 

100%|███████████████████████████████████████████████████████████████████████████████| 37/37 [03:40<00:00,  5.96s/it]


Time taken:  4686.438013553619


IndexError    2
Name: reason, dtype: int64

NoSuchElementException    8
Name: reason, dtype: int64

AssertionError: 

In [44]:
er_df_1_2

,code,reason
0,5293,IndexError
1,8664,IndexError


In [42]:
er_df_2

,code,reason
0,2836,NoSuchElementException
1,5797,NoSuchElementException
2,5141,NoSuchElementException
3,0128,NoSuchElementException
4,5295,NoSuchElementException
5,5075,NoSuchElementException
6,9954,NoSuchElementException
7,2429,NoSuchElementException


In [132]:
data1_2 = pd.concat([pd.read_csv(file) for file in glob('temp_data/data1*.csv')])
data1_2['code'] = data1_2.code.apply(lambda x: str(x).zfill(4))
data1_2.to_csv(out_dir+'klse_01b_latest_data.csv', index=False)
print(data1_2.shape)

data1 = data1_1.merge(data1_2,'inner')
data1 = data1[['code','category','Price','52w low','52w high','BVPS','P/E','EPS','EY','DPS','DY','ROE','ROE2','D/E','D/E2']]
print(data1.shape)
data1.to_csv(out_dir+'klse_01_latest_data.csv', index=False)
display(data1.head(1))

(785, 4)
(785, 15)


,code,category,Price,52w low,52w high,NTA,P/E,EPS,EY,DPS,DY,ROE,ROE2,D/E,D/E2
0,5250,Consumer Products & Services,1.3,1.21,1.56,0.0893,30.0,4.38,3.37,2.3,1.77,49.05,61.69,6.06,0.48


In [133]:
data2 = pd.concat([pd.read_csv(file) for file in glob('temp_data/data2*.csv')])
data2['code'] = data2.code.apply(lambda x: str(x).zfill(4))
data2.to_csv(out_dir+'klse_02_10yr_data.csv', index=False)
print(data2.shape)
display(data2.head(1))

(17270, 12)


,code,detail,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10
0,5250,Year End,2018-12,2017-12,2016-12,2015-12,2014-12,2013-12,2012-12,2011-12,2010-12,2009-12


In [139]:
balance = data1_1[~data1_1.code.isin(data2.code)]
print(balance.shape)
balance

(2, 12)


,code,category,Price,52w low,52w high,NTA,P/E,EPS,EY,DPS,DY,ROE
31,5293,Construction,1.62,1.010,2.050,1.4500,11.0,15.28,9.43,0.0,0.0,10.54
317,5295,UNKNOWN,0.40,0.242,0.716,0.1382,9.0,4.48,11.20,1.0,2.5,32.42


In [100]:
er_df_1, er_df_2 = process_stage_2(balance, 20)
display(er_df_1.reason.value_counts())
display(er_df_2.reason.value_counts())

assert(len(er_df_1) == 0) 
assert(len(er_df_2) == 0) 

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [00:57<00:00,  5.75s/it]


Time taken:  68.09922051429749


IndexError    1
Name: reason, dtype: int64

NoSuchElementException    1
Name: reason, dtype: int64

AssertionError: 

In [110]:
er_df_1

,code,reason
0,5293,IndexError


In [111]:
er_df_2

,code,reason
0,5295,NoSuchElementException


In [140]:
data1.code.nunique()

785

In [141]:
data2.code.nunique()

785

### Stage 3

In [3]:
def get_min_gr(y3, y5, y10):
    y = [y3, y5, y10]
    y = [x for x in y if x > 0]
    if len(y) == 0:
        y = [0.0]
    return min(y)

In [4]:
error3, result3, symbol= [], [], ['—','\xa0-\xa0']
for i in tqdm(data2.code.unique()):
    stock      = data2[data2.code == str(i).zfill(4)]
    net_profit = [int(x.replace(',','')) for x in stock[stock.detail=='NP'].values[0][2:] if x not in symbol]
    nYear      = len(net_profit)
    if nYear >= 5:
#         try:
        temp           = [0 if x > 0 else 1 for x in net_profit]
        np_check       = 1 if sum(temp) == 0 else 0 # All positive net profit
        neg_np         = '+' if np_check == 1 else '+'.join([str(n+1) for n, x in enumerate(temp) if x == 1]) # Position of negative net profit
        
        ocf            = [int(x.replace(',','')) if x not in symbol else 0 for x in stock[stock.detail=='OCF'].values[0][2:2+nYear]]
        ocf_start      = ocf[-1]
        ocf_end        = ocf[0]
        temp           = [0 if x > 0 else 1 for x in ocf]
        ocf_check      = 1 if sum(temp) == 0 else 0
        neg_ocf        = '+' if np_check == 1 else '+'.join([str(n+1) for n, x in enumerate(temp) if x == 1])

        rvnu           = [int(x.replace(',','')) if x not in symbol else 0 for x in stock[stock.detail=='Revenue'].values[0][2:2+nYear]]
        rvnu_start     = rvnu[-1]
        rvnu_end       = rvnu[0]
        rvnu_gr        = [float(x.replace(',','')) if x not in symbol else 0.0 for x in stock[stock.detail=='Revenue GR'].values[0][2:2+nYear]]
        temp           = [0 if x > 0 else 1 for x in rvnu_gr]
        rvnu_gr_check  = 1 if sum(temp) == 0 else 0
        neg_rvnu_gr    = '+' if rvnu_gr_check == 1 else '+'.join([str(n+1) for n, x in enumerate(temp) if x == 1])

        nosh           = [int(x.replace(',','')) if x not in symbol else 0 for x in stock[stock.detail=='Shares'].values[0][2:2+nYear]]
        price          = [float(str(x).replace(',','')) if x not in symbol else 0.0 for x in stock[stock.detail=='Price'].values[0][2:2+nYear]]

        np_margin      = [float(x.replace(',','')) for x in stock[stock.detail=='NP Margin'].values[0][2:] if x not in symbol]
        np_margin_avg  = np.around(np.mean(np_margin), 2)

        eps            = [float(x.replace(',','')) if x not in symbol else 0.0 for x in stock[stock.detail=='EPS'].values[0][2:2+nYear]]
        eps_start      = eps[-1]
        eps_end        = eps[0]
        eps_gr         = [float(x.replace(',','')) if x not in symbol else 0.0 for x in stock[stock.detail=='EPS GR'].values[0][2:2+nYear]]
        temp           = [0 if x > 0 else 1 for x in eps_gr]
        eps_gr_check   = 1 if sum(temp) == 0 else 0
        neg_eps_gr     = '+' if eps_gr_check == 1 else '+'.join([str(n+1) for n, x in enumerate(temp) if x == 1])
#         eps_gr_avg     = np.around(np.mean(eps_gr), 2)
#         eps_cagr       = np.around(((eps[0]/eps[-1])**(1/(len(eps)-1))-1)*100, 2)
        eps_gr_3y_avg  = [float(x.replace(',','')) if x not in symbol else 0.0 for x in stock[stock.detail=='EPS GR 3Y avg'].values[0][2:2+nYear]]
        eps_gr_5y_avg  = [float(x.replace(',','')) if x not in symbol else 0.0 for x in stock[stock.detail=='EPS GR 5Y avg'].values[0][2:2+nYear]]
        eps_gr_10y_avg = [float(x.replace(',','')) if x not in symbol else 0.0 for x in stock[stock.detail=='EPS GR 10Y avg'].values[0][2:2+nYear]]
        if len(eps_gr_3y_avg) == 0:
            eps_gr_3y_avg = [0.0]
        if len(eps_gr_5y_avg) == 0:
            eps_gr_5y_avg = [0.0]
        if len(eps_gr_10y_avg) == 0:
            eps_gr_10y_avg = [0.0]
        eps_gr_min     = get_min_gr(eps_gr_3y_avg[0],eps_gr_5y_avg[0],eps_gr_10y_avg[0])

        ey             = [np.around(eps[x]/price[x]*100, 2) for x in range(len(price)) if price[x] != 0]# EPS / Price
        ey_avg         = np.around(np.mean(ey),2)
        ey_3y_avg      = np.around(np.mean(ey[:3]), 2) if len(ey) >= 3 else 0.0
        ey_5y_avg      = np.around(np.mean(ey[:5]), 2) if len(ey) >= 5 else 0.0
        ey_10y_avg     = np.around(np.mean(ey), 2) if len(ey) == 10 else 0.0
        
        pe             = [np.around(price[x]/eps[x], 2) for x in range(len(price)) if eps[x] != 0]# Price / EPS
        pe_min         = min(pe)
        pe_max         = max(pe)
        pe_mid         = np.around((pe_max + pe_min)/2, 2)
        pe_avg         = np.around(np.mean(pe), 2)

        dpayout        = [float(x.replace(',','')) if x not in symbol else 0.0 for x in stock[stock.detail=='Div Payout'].values[0][2:2+nYear]]
        if len(dpayout) == 0:
            dpayout = [0.0]
        dpayout_avg    = np.around(np.mean(dpayout), 2)

        dps            = [float(x.replace(',','')) if x not in symbol else 0.0 for x in stock[stock.detail=='DPS'].values[0][2:2+nYear]]
        if len(dps) == 0:
            dps = [0.0]
        dy             = [np.around(dps[x]/price[x]*100, 2) for x in range(len(price)) if price[x] != 0]# DPS / Price
        dy_avg         = np.around(np.mean(dy),2)
        dy_3y_avg      = np.around(np.mean(dy[:3]), 2) if len(dy) >= 3 else 0.0
        dy_5y_avg      = np.around(np.mean(dy[:5]), 2) if len(dy) >= 5 else 0.0
        dy_10y_avg     = np.around(np.mean(dy), 2) if len(dy) == 10 else 0.0

        dates          = [datetime.strptime(x, '%Y-%m') for x in stock[stock.detail=='Year End'].values[0][2:] if x not in symbol]
        annual         = dates[0]      

        result3.append([str(i), annual, nYear, np_margin_avg,  
                        pe_min, pe_max, pe_mid, pe_avg,
                        eps_gr_min,eps_gr_3y_avg[0], eps_gr_5y_avg[0], eps_gr_10y_avg[0], 
                        ey_avg, ey_3y_avg, ey_5y_avg, ey_10y_avg, 
                        dy_avg, dy_3y_avg, dy_5y_avg, dy_10y_avg, dpayout_avg,
                        np_check, eps_gr_check, ocf_check, neg_rvnu_gr, neg_np, neg_eps_gr, neg_ocf,
                        rvnu_start, rvnu_end, eps_start, eps_end, ocf_start, ocf_end])
#         except:
#             error3.append([str(i), 'Calculation Error'])
    else:
        error3.append([str(i), 'Less than 5 years'])

100%|████████████████████████████████████████████████████████████████████████████| 785/785 [00:05<00:00, 137.24it/s]


In [5]:
er_df_3 = pd.DataFrame(error3, columns=['code','reason'])
display(er_df_3.reason.value_counts())

data3 = pd.DataFrame(result3, columns=['code','Annual','nYear','NPM avg',
                                       'P/E min','P/E max','P/E mid','P/E avg',
                                       'EPS GR min','EPS GR 3Y avg','EPS GR 5Y avg','EPS GR 10Y avg',
                                       'EY avg','EY 3Y avg','EY 5Y avg','EY 10Y avg',
                                       'DY avg','DY 3Y avg','DY 5Y avg','DY 10Y avg','DPayout avg',
                                       'check NP','check EPS GR','check OCF','neg Revenue','neg NP','neg EPS GR','neg OCF',
                                       'Revenue start','Revenue end','EPS start','EPS end','OCF start','OCF end'])
data3 = data3.merge(stock_code,'left')
data3 = data3.merge(data1,'left')
temp = data3.columns.tolist()
data3 = data3[temp[:1]+temp[-15:-13]+temp[1:3]+temp[-13:]+temp[3:-15]]
# data3 = data3[data3['check NP']==1]
# data3 = data3[data3['check OCF']==1]
print(data3.shape)
data3.sort_values(['category','code'],ascending=False,inplace=True)
data3.reset_index(drop=True,inplace=True)
data3.to_csv(out_dir+'klse_03_stage_3.csv',index=False)
display(data3.head(1))

Less than 5 years    11
Name: reason, dtype: int64

(774, 49)


,code,name,category,Annual,nYear,Price,52w low,52w high,NTA,P/E,...,neg Revenue,neg NP,neg EPS GR,neg OCF,Revenue start,Revenue end,EPS start,EPS end,OCF start,OCF end
0,8567,SALCON BERHAD,Utilities,2018-12-01,10,0.175,0.1,0.28,0.52,-17.0,...,1+3+4+6+7+8+10,2,1+2+5+7+8+10,1+2+3+5+6+7+8,370,130,0.04,0.0,44,-30


In [7]:
error3

[['5281', 'Less than 5 years'],
 ['5269', 'Less than 5 years'],
 ['5273', 'Less than 5 years'],
 ['5229', 'Less than 5 years'],
 ['5291', 'Less than 5 years'],
 ['5280', 'Less than 5 years'],
 ['6633', 'Less than 5 years'],
 ['5286', 'Less than 5 years'],
 ['5256', 'Less than 5 years'],
 ['5270', 'Less than 5 years'],
 ['5292', 'Less than 5 years']]

In [6]:
display(data3['check NP'].value_counts())
display(data3['check EPS GR'].value_counts())
display(data3['check OCF'].value_counts())

0    427
1    347
Name: check NP, dtype: int64

0    772
1      2
Name: check EPS GR, dtype: int64

0    571
1    203
Name: check OCF, dtype: int64

In [8]:
data3.columns

Index(['code', 'name', 'category', 'Annual', 'nYear', 'Price', '52w low',
       '52w high', 'NTA', 'P/E', 'EPS', 'EY', 'DPS', 'DY', 'ROE', 'ROE2',
       'D/E', 'D/E2', 'NPM avg', 'P/E min', 'P/E max', 'P/E mid', 'P/E avg',
       'EPS GR min', 'EPS GR 3Y avg', 'EPS GR 5Y avg', 'EPS GR 10Y avg',
       'EY avg', 'EY 3Y avg', 'EY 5Y avg', 'EY 10Y avg', 'DY avg', 'DY 3Y avg',
       'DY 5Y avg', 'DY 10Y avg', 'DPayout avg', 'check NP', 'check EPS GR',
       'check OCF', 'neg Revenue', 'neg NP', 'neg EPS GR', 'neg OCF',
       'Revenue start', 'Revenue end', 'EPS start', 'EPS end', 'OCF start',
       'OCF end'],
      dtype='object')

In [9]:
data3[data3.Price > data3.BVPS]

,code,name,category,Annual,nYear,Price,52w low,52w high,NTA,P/E,...,neg Revenue,neg NP,neg EPS GR,neg OCF,Revenue start,Revenue end,EPS start,EPS end,OCF start,OCF end
1,8524,TALIWORKS CORPORATION BERHAD,Utilities,2019-12-01,10,0.855,0.635,0.980,0.5038,21.0,...,4+9,+,1+3+5+7+10,+,172,360,0.02,0.04,50,643
4,6033,PETRONAS GAS BERHAD,Utilities,2019-12-01,10,15.280,13.360,18.388,6.6939,16.0,...,1+9,+,4+6+9,+,3525,5458,0.72,0.97,0,3357
5,5347,TENAGA NASIONAL BHD,Utilities,2019-12-01,10,12.200,10.365,13.704,10.1957,15.0,...,+,+,2+3+5+9,+,30320,50939,0.59,0.80,7970,19458
6,5272,RANHILL HOLDINGS BERHAD,Utilities,2019-12-01,10,0.995,0.820,1.440,0.5500,13.0,...,6+7+10,6+7+8+9,2+3+5+6+7+8+9+10,10,171,1630,0.00,0.07,0,350
8,5209,GAS MALAYSIA BERHAD,Utilities,2019-12-01,10,2.810,2.300,2.960,0.7997,18.0,...,+,+,2+5+6+7+8+9,+,1807,6886,0.23,0.15,369,119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,5253,ECONPILE HOLDINGS BERHAD,Construction,2019-06-01,7,0.500,0.300,0.880,0.3000,11.0,...,1+7,+,1+7,+,386,663,0.02,0.01,27,30
761,5085,MUDAJAYA GROUP BERHAD,Construction,2018-12-01,10,0.305,0.145,0.555,0.2500,-2.0,...,2+4+5+6+10,1+2+3+4+5,1+2+3+4+5+6+10,4+5+6,720,760,0.24,-0.67,221,34
765,5006,STELLA HOLDINGS BERHAD,Construction,2019-03-01,10,0.750,0.650,1.350,0.6700,-13.0,...,2+3+4+5+9+10,1+2+9,1+2+3+4+8+9+10,1+2+6+8+10,96,55,0.10,-0.07,0,-3
770,2283,ZELAN BERHAD,Construction,2018-12-01,10,0.065,0.030,0.110,0.0600,21.0,...,2+3+6+8+9+10,1+2+3+6+8+9+10,1+2+3+4+5+6+7+8+9+10,1+2+3+4+5+6+7+8+9+10,2008,72,-0.20,-0.03,-175,-4


### Stage 4

In [10]:
data4 = data3[['code','name','Price','52w low','52w high','EPS GR min','P/E mid','DPayout avg']].copy()
result4 = []
for n, x in tqdm(data4.iterrows(), total=len(data4)):
    price_10y = [np.around(x['Price'] * ((1 + (x['EPS GR min'] / 100))**i) ,2) for i in range(5)]
    total_eps = np.around(sum(price_10y), 2)
    expected_price = np.around(price_10y[-1] * x['P/E mid'],2)
    total_dvd = np.around(total_eps * (x['DPayout avg'] / 100), 2)
    total_return = np.around(total_eps + total_dvd, 2)
    intrinsic = np.around(total_return / ((1 + (x['EPS GR min'] / 100))**5), 2)
    price_A = np.around(intrinsic * 0.75, 2)
    price_B = np.around((x['52w high'] - x['52w low']) * 0.33 + x['52w low'], 2)
    result4.append([str(x['code']), intrinsic, price_A, price_B])

data4 = data4.merge(pd.DataFrame(result4, columns=['code','intrinsic','Price_A','Price_B']))
print(data4.shape)
data4.to_csv(out_dir+'klse_04_intrinsic.csv',index=False)
data4.head(1)

100%|███████████████████████████████████████████████████████████████████████████| 774/774 [00:00<00:00, 3401.60it/s]


(774, 11)


,code,name,Price,52w low,52w high,EPS GR min,P/E mid,DPayout avg,intrinsic,Price_A,Price_B
0,8567,SALCON BERHAD,0.175,0.1,0.28,0.0,27.08,16.52,1.05,0.79,0.16


In [11]:
data4.columns

Index(['code', 'name', 'Price', '52w low', '52w high', 'EPS GR min', 'P/E mid',
       'DPayout avg', 'intrinsic', 'Price_A', 'Price_B'],
      dtype='object')

In [12]:
# Stocks which have prices lower than fair value price B
data4[data4.Price < data4.Price_B]

,code,name,Price,52w low,52w high,EPS GR min,P/E mid,DPayout avg,intrinsic,Price_A,Price_B
6,5272,RANHILL HOLDINGS BERHAD,0.995,0.820,1.44,0.44,15.00,25.66,6.17,4.63,1.02
10,4677,YTL CORPORATION BERHAD,0.735,0.600,1.17,0.00,33.40,69.97,6.29,4.72,0.79
12,9806,NATIONWIDE EXPRESS HOLDINGS BERHAD,0.100,0.040,0.28,0.00,14.50,76.42,0.88,0.66,0.12
14,8346,PERAK CORPORATION BERHAD,0.175,0.090,0.47,0.00,2.75,5.94,0.95,0.71,0.22
15,8133,BOUSTEAD HEAVY INDUSTRIES CORPORATION BHD,0.630,0.560,1.36,0.00,106.79,5.09,3.31,2.48,0.82
...,...,...,...,...,...,...,...,...,...,...,...
764,5042,TSR CAPITAL BERHAD,0.230,0.180,0.49,0.00,38.96,0.00,1.15,0.86,0.28
765,5006,STELLA HOLDINGS BERHAD,0.750,0.650,1.35,0.00,-2.93,0.00,3.75,2.81,0.88
767,3565,WCE HOLDINGS BERHAD,0.260,0.145,0.63,0.00,3.88,0.00,1.30,0.98,0.31
769,3204,GEORGE KENT (MALAYSIA) BERHAD,0.655,0.430,1.27,8.14,19.44,46.92,3.85,2.89,0.71


### Stage 5

Earning Yield Level:
    - 5  PE <=6.67    EY >=15%         DY >=3%
    - 4  PE >6.67-10  EY 10%-<15%      DY 2.5%-<3%
    - 3  PE >10-20    EY 5%-<10%       DY 2%-<2.5%
    - 2  PE >20-30    EY 3.33%-<5%     DY 1.5%-<2%
    - 1  PE >30-40    EY 2.5%-<3.33%   DY 1%-<1.5%
    - 0  PE >4        EY <2.5%         DY <1%

In [13]:
data5 = data3[['code','nYear','neg Revenue','neg EPS GR','neg OCF','Revenue start','Revenue end','EPS start','EPS end',
               'OCF start','OCF end','ROE','ROE2','D/E','D/E2','P/E mid','EPS','EPS GR min']].copy()
ey_level = [min(data3['EY']),2.5,3.33,5,10,15,max(data3['EY'])+1]
dy_level = [min(data3['DY']),1,1.5,2,2.5,3,max(data3['DY'])+1]
ey_label = [0,1,2,3,4,5]
data5['EY level'] = pd.cut(data3['EY'], ey_level, False, ey_label)
data5['DY level'] = pd.cut(data3['DY'], dy_level, False, ey_label)
data5['neg Revenue count'] = data5['neg Revenue'].apply(lambda x: len(x.split('+')) if x != '+' else 0)
data5['neg EPS count'] = data5['neg EPS GR'].apply(lambda x: len(x.split('+')) if x != '+' else 0)
data5['neg OCF count'] = data5['neg OCF'].apply(lambda x: len(x.split('+')) if x != '+' else 0)
data5['Revenue Growth'] = data5.apply(lambda x: 1 if x['Revenue end']>=x['Revenue start'] and x['neg Revenue count']==0 else
                                        0.5 if x['Revenue end']>=x['Revenue start'] and x['neg Revenue count']<=3 else 0, axis=1)
data5['EPS Growth'] = data5.apply(lambda x: 1 if x['EPS end']>=x['EPS start'] and x['neg EPS count']==0 else
                                    0.5 if x['EPS end']>=x['EPS start'] and x['neg EPS count']<=2 else 0, axis=1)
data5['OCF Growth'] = data5.apply(lambda x: 1 if x['OCF end']>=x['OCF start'] and x['neg OCF count']==0 else
                                    0.5 if x['OCF end']>=x['OCF start'] and x['neg OCF count']<=3 else 0, axis=1)
data5['ROE Health'] = data5.apply(lambda x: 1 if x['ROE']>=15 else 0.5 if x['ROE2']>=15 else 0, axis=1)
data5['D/E Health'] = data5.apply(lambda x: 1 if x['D/E']<=0.5 else 0.5 if x['D/E2']<=0.5 else 0, axis=1)
data5['3G2H'] = data5['Revenue Growth']+data5['EPS Growth']+data5['OCF Growth']+data5['ROE Health']+data5['D/E Health']
data5['PE Value'] = np.around(data5['P/E mid']*data5['EPS'], 2)
data5['EPS GR Value'] = np.around(data5['EPS']*data5['EPS GR min'], 2)
data5.drop(columns=['neg Revenue','neg EPS GR','neg OCF'], inplace=True)
data5.sort_values('3G2H', inplace=True, ascending=False)
data5.reset_index(drop=True, inplace=True)
print(data5.shape)
data5.to_csv(out_dir+'klse_05_3G2H_score_with_valuation.csv',index=False)
data5.head(1)

(774, 28)


,code,nYear,Revenue start,Revenue end,EPS start,EPS end,OCF start,OCF end,ROE,ROE2,...,neg EPS count,neg OCF count,Revenue Growth,EPS Growth,OCF Growth,ROE Health,D/E Health,3G2H,PE Value,EPS GR Value
0,4731,10,695,3246,0.14,0.67,78,557,16.32,16.73,...,0,0,1.0,1.0,1.0,1.0,1.0,5.0,1306.64,592.17


In [14]:
data5.columns

Index(['code', 'nYear', 'Revenue start', 'Revenue end', 'EPS start', 'EPS end',
       'OCF start', 'OCF end', 'ROE', 'ROE2', 'D/E', 'D/E2', 'P/E mid', 'EPS',
       'EPS GR min', 'EY level', 'DY level', 'neg Revenue count',
       'neg EPS count', 'neg OCF count', 'Revenue Growth', 'EPS Growth',
       'OCF Growth', 'ROE Health', 'D/E Health', '3G2H', 'PE Value',
       'EPS GR Value'],
      dtype='object')

### Final Table

In [15]:
final = data3.merge(data4.iloc[:,[0]+list(range(-3,0))])\
             .merge(data5.iloc[:,[0]+list(range(-8,0))])
temp = final.columns.tolist()
final = final[temp[:5]+temp[-3:]+temp[-11:-8]+temp[5:-11]+temp[-8:-3]]
print(final.shape)
final.to_csv(out_dir+'klse_06_final_table.csv',index=False)
final.head(1)

(774, 60)


,code,name,category,Annual,nYear,3G2H,PE Value,EPS GR Value,intrinsic,Price_A,...,Revenue end,EPS start,EPS end,OCF start,OCF end,Revenue Growth,EPS Growth,OCF Growth,ROE Health,D/E Health
0,8567,SALCON BERHAD,Utilities,2018-12-01,10,1.0,-27.62,-0.0,1.05,0.79,...,130,0.04,0.0,44,-30,0.0,0.0,0.0,0.0,1.0


In [16]:
display(final[final.code=='5398'].iloc[:,:18])
display(final[final.code=='5398'].iloc[:,18:33])
display(final[final.code=='5398'].iloc[:,33:48])
display(final[final.code=='5398'].iloc[:,48:])

,code,name,category,Annual,nYear,3G2H,PE Value,EPS GR Value,intrinsic,Price_A,Price_B,Price,52w low,52w high,NTA,P/E,EPS,EY
751,5398,GAMUDA BERHAD,Construction,2019-07-01,10,2.5,498.72,119.74,22.51,16.88,3.0,3.27,2.36,4.3,3.34,12.0,28.24,8.64


,DPS,DY,ROE,ROE2,D/E,D/E2,NPM avg,P/E min,P/E max,P/E mid,P/E avg,EPS GR min,EPS GR 3Y avg,EPS GR 5Y avg,EPS GR 10Y avg
751,12.0,3.67,8.46,9.03,0.37,0.37,19.61,13.25,22.08,17.66,17.6,4.24,4.24,-1.48,11.51


,EY avg,EY 3Y avg,EY 5Y avg,EY 10Y avg,DY avg,DY 3Y avg,DY 5Y avg,DY 10Y avg,DPayout avg,check NP,check EPS GR,check OCF,neg Revenue,neg NP,neg EPS GR
751,5.84,5.92,5.79,5.84,2.89,2.86,2.71,2.89,55.69,1,0,0,4+6+10,+,2+3+4+5+7


,neg OCF,Revenue start,Revenue end,EPS start,EPS end,OCF start,OCF end,Revenue Growth,EPS Growth,OCF Growth,ROE Health,D/E Health
751,+,2455,4565,0.16,0.28,416,485,0.5,0.0,1.0,0.0,1.0
